In [67]:
import boto3
import json
import time

# LocalStack endpoint (accessible from host)
ENDPOINT_URL = "http://localhost:4566"
REGION = "us-east-1"

# Resource Names
TASK_QUEUE_NAME = "task-queue"
NOTIFY_QUEUE_NAME = "notification-queue"
DLQ_QUEUE_NAME = "dlq-queue"
BUCKET_NAME = "results-bucket"

sqs = boto3.client("sqs", endpoint_url=ENDPOINT_URL, region_name=REGION)
s3 = boto3.client("s3", endpoint_url=ENDPOINT_URL, region_name=REGION)
lambdas = boto3.client("lambda", endpoint_url=ENDPOINT_URL, region_name=REGION)
iam = boto3.client("iam", endpoint_url=ENDPOINT_URL, region_name=REGION)


In [69]:
def create_or_get_queue(queue_name):
    try:
        return sqs.create_queue(QueueName=queue_name)['QueueUrl']
    except:
        return sqs.get_queue_url(QueueName=queue_name)['QueueUrl']

def get_queue_arn(queue_url):
    return sqs.get_queue_attributes(QueueUrl=queue_url, AttributeNames=['QueueArn'])['Attributes']['QueueArn']

print("🏗️ Creating Infrastructure...")

try:
    s3.create_bucket(Bucket=BUCKET_NAME)
    print(f"✅ S3 Bucket: {BUCKET_NAME}")
except:
    print(f"✅ S3 Bucket: {BUCKET_NAME} (exists)")

TASK_QUEUE_URL = create_or_get_queue(TASK_QUEUE_NAME)
NOTIFY_QUEUE_URL = create_or_get_queue(NOTIFY_QUEUE_NAME)
DLQ_URL = create_or_get_queue(DLQ_QUEUE_NAME)
print(f"✅ Queues created")

try:
    ROLE_ARN = iam.create_role(
        RoleName="lambda-role",
        AssumeRolePolicyDocument=json.dumps({
            "Version": "2012-10-17",
            "Statement": [{
                "Effect": "Allow",
                "Principal": {"Service": "lambda.amazonaws.com"},
                "Action": "sts:AssumeRole"
            }]
        })
    )['Role']['Arn']
except:
    ROLE_ARN = iam.get_role(RoleName="lambda-role")['Role']['Arn']
print(f"✅ IAM Role created")

sqs.set_queue_attributes(
    QueueUrl=TASK_QUEUE_URL,
    Attributes={
        'RedrivePolicy': json.dumps({
            'deadLetterTargetArn': get_queue_arn(DLQ_URL),
            'maxReceiveCount': '2'
        })
    }
)
print(f"✅ DLQ configured\n")


🏗️ Creating Infrastructure...
✅ S3 Bucket: results-bucket
✅ Queues created
✅ IAM Role created
✅ DLQ configured



In [71]:
import zipfile
import io
import os

def create_lambda_zip():
    zip_buffer = io.BytesIO()
    with zipfile.ZipFile(zip_buffer, "a", zipfile.ZIP_DEFLATED, False) as zip_file:
        print("📦 Packaging files for Lambda...\n")
        
        # Add app folder (including nested folders)
        for root, dirs, files in os.walk('../app'):
            for file in files:
                if file.endswith('.py'):
                    file_path = os.path.join(root, file)
                    arcname = os.path.relpath(file_path, start='..')
                    zip_file.write(file_path, arcname)
                    print(f"  ✅ {arcname}")
        
        # Add lambdas
        for root, dirs, files in os.walk('../lambdas'):
            for file in files:
                if file.endswith('.py'):
                    file_path = os.path.join(root, file)
                    zip_file.write(file_path, arcname=file)
                    print(f"  ✅ {file}")
        
        print(f"\n📊 Total size: {zip_buffer.tell()} bytes\n")
    return zip_buffer.getvalue()

zip_content = create_lambda_zip()

for func in ["task_lambda", "notification_lambda", "dlq_processor_lambda"]:
    try: lambdas.delete_function(FunctionName=func)
    except: pass

LAMBDA_ENV = {
    'Variables': {
        'AWS_ENDPOINT_URL': 'http://localstack:4566',
        'AWS_REGION': REGION
    }
}

lambdas.create_function(
    FunctionName="task_lambda",
    Runtime="python3.10",
    Role=ROLE_ARN,
    Handler="task_lambda.lambda_handler",
    Code={'ZipFile': zip_content},
    Environment=LAMBDA_ENV,
    Timeout=30
)

lambdas.create_function(
    FunctionName="notification_lambda",
    Runtime="python3.10",
    Role=ROLE_ARN,
    Handler="notification_lambda.lambda_handler",
    Code={'ZipFile': zip_content},
    Environment=LAMBDA_ENV,
    Timeout=30
)

lambdas.create_function(
    FunctionName="dlq_processor_lambda",
    Runtime="python3.10",
    Role=ROLE_ARN,
    Handler="dlq_processor_lambda.lambda_handler",
    Code={'ZipFile': zip_content},
    Environment=LAMBDA_ENV,
    Timeout=30
)

print("✅ Lambdas deployed!\n")


📦 Packaging files for Lambda...

  ✅ app\auth.py
  ✅ app\config.py
  ✅ app\database.py
  ✅ app\notifier.py
  ✅ app\processors.py
  ✅ app\storage.py
  ✅ app\__init__.py
  ✅ app\helpers\discount_calculator.py
  ✅ app\helpers\__init__.py
  ✅ api_lambda.py
  ✅ authorizer_lambda.py
  ✅ dlq_processor_lambda.py
  ✅ notification_lambda.py
  ✅ task_lambda.py

📊 Total size: 7918 bytes

✅ Lambdas deployed!



In [72]:
def add_trigger(queue_url, function_name):
    try:
        queue_arn = sqs.get_queue_attributes(QueueUrl=queue_url, AttributeNames=['QueueArn'])['Attributes']['QueueArn']
        lambdas.create_event_source_mapping(
            EventSourceArn=queue_arn,
            FunctionName=function_name,
            BatchSize=10
        )
        print(f"🔗 Linked: {function_name}")
    except:
        print(f"🔗 Already linked: {function_name}")

add_trigger(TASK_QUEUE_URL, "task_lambda")
add_trigger(NOTIFY_QUEUE_URL, "notification_lambda")
add_trigger(DLQ_URL, "dlq_processor_lambda")


🔗 Already linked: task_lambda
🔗 Already linked: notification_lambda
🔗 Already linked: dlq_processor_lambda


In [75]:
import time
import json
import base64

order_id = f"ORD-{int(time.time())}"

order_message = {
    "order_id": order_id,
    "items": [
        {"name": "Laptop", "price": 999.99, "quantity": 1},
        {"name": "Mouse", "price": 29.99, "quantity": 2}
    ],
    "promo_code": "SAVE10"
}

sqs.send_message(QueueUrl=TASK_QUEUE_URL, MessageBody=json.dumps(order_message))

print(f"📤 Order sent: {order_id}")
print("⏳ Waiting 10s...\n")
time.sleep(10)

# Check S3 result
try:
    obj = s3.get_object(Bucket=BUCKET_NAME, Key=f"{order_id}.json")
    invoice = json.loads(obj['Body'].read())
    print("✅ SUCCESS!")
    print(f"   Subtotal: ${invoice['subtotal']:.2f}")
    print(f"   Discount: ${invoice['discount']:.2f}")
    print(f"   Final: ${invoice['final_total']:.2f}")
    
    # Check nested folder values
    if 'bulk_discount' in invoice:
        print(f"\n🧪 NESTED FOLDER TEST:")
        print(f"   Bulk Discount: ${invoice['bulk_discount']:.2f}")
        print(f"   Tax: ${invoice['tax']:.2f}")
        print(f"   ✅ app/helpers/discount_calculator.py ACCESSED!")
    
except Exception as e:
    print(f"❌ Error: {e}")

# Check Lambda logs
print("\n📋 Checking Lambda logs...")
try:
    import subprocess
    result = subprocess.run(
        ['docker', 'logs', 'localstack_main', '--tail', '50'],
        capture_output=True,
        text=True
    )
    
    # Filter for our order
    for line in result.stdout.split('\n'):
        if order_id in line or 'NESTED FOLDER TEST' in line or 'bulk_discount' in line:
            print(f"  {line}")
except:
    print("  (Run: docker logs localstack_main --tail 50)")


📤 Order sent: ORD-1770707423
⏳ Waiting 10s...

✅ SUCCESS!
   Subtotal: $1059.97
   Discount: $106.00
   Final: $953.97

🧪 NESTED FOLDER TEST:
   Bulk Discount: $159.00
   Tax: $76.32
   ✅ app/helpers/discount_calculator.py ACCESSED!

📋 Checking Lambda logs...


In [74]:
import time
import json

# Order Input
order_id = f"ORD-{int(time.time())}"

order_message = {
    "order_id": order_id,
    "items": [
        {"name": "Laptop", "price": 200.99, "quantity": 1},
        {"name": "Mouse", "price": 30.99, "quantity": 2},
        {"name": "Keyboard", "price": 50.00, "quantity": 1}


    ],
    "promo_code": "SAVE10"
}

sqs.send_message(
    QueueUrl=TASK_QUEUE_URL,
    MessageBody=json.dumps(order_message)
)

print(f"📤 Order sent: {order_id}")
print(f"   Items: {len(order_message['items'])} items")
print(f"   Promo: {order_message['promo_code']}")
print("⏳ Waiting for processing (10s)...")
time.sleep(10)

# Check S3 for invoice
try:
    obj = s3.get_object(Bucket=BUCKET_NAME, Key=f"{order_id}.json")
    invoice = json.loads(obj['Body'].read())
    print(f"\n INVOICE FOUND in S3:")
    print(f"  Order ID: {invoice['order_id']}")
    print(f"  Subtotal: ${invoice['subtotal']}")
    print(f"  Discount: ${invoice['discount']}")
    print(f"  Final Total: ${invoice['final_total']}")
    print(f"  Status: {invoice['status']}")
except Exception as e:
    print(f" Invoice not found: {e}")



📤 Order sent: ORD-1770707404
   Items: 3 items
   Promo: SAVE10
⏳ Waiting for processing (10s)...

 INVOICE FOUND in S3:
  Order ID: ORD-1770707404
  Subtotal: $312.97
  Discount: $31.3
  Final Total: $281.67
  Status: completed


In [38]:
# logs (application logs), request -> use models (validate it ) , Authenication JWT  (any other inputs also fine) add the correlaeation Id ()  try to send multiple msgs to the queue ( inputs)

In [ ]:
#DL queue -> DL queue , identify  the sqs message -> how does the message will trigger the lambda function ->, sqs trigger lambda queue please check it , reseacch about the DLQs ,  

In [ ]:
#  make some modules and check it ,  

In [ ]:
# name the handler function (test it ) ->if any other names and check it once   , authentication : request -> BFF(another microservice) -> token -> (message ) -> Lambda function

In [66]:
# Cell 7: Bulk Test with 1 Failed Order → DLQ

import time
import json

# Configure fast retries
sqs.set_queue_attributes(QueueUrl=TASK_QUEUE_URL, Attributes={'VisibilityTimeout': '5'})

print("📦 Sending 10 Orders (9 good + 1 bad)...")
print("="*60)

# 9 good orders + 1 bad order (negative price)
bulk_orders = [
    {"items": [{"name": "Laptop", "price": 999.99, "quantity": 1}], "promo_code": "SAVE10"},
    {"items": [{"name": "Monitor", "price": 299.99, "quantity": 2}], "promo_code": "SAVE20"},
    {"items": [{"name": "Keyboard", "price": 79.99, "quantity": 1}], "promo_code": None},
    {"items": [{"name": "Mouse", "price": -49.99, "quantity": -1}], "promo_code": "SAVE10"},  # BAD ORDER
    {"items": [{"name": "Headphones", "price": 149.99, "quantity": 1}], "promo_code": "SAVE10"},
    {"items": [{"name": "Webcam", "price": 89.99, "quantity": 1}], "promo_code": "FREESHIP"},
    {"items": [{"name": "Desk Chair", "price": 249.99, "quantity": 1}], "promo_code": "SAVE30"},
    {"items": [{"name": "USB Hub", "price": 39.99, "quantity": 2}], "promo_code": "SAVE10"},
    {"items": [{"name": "Speaker", "price": 79.99, "quantity": 1}], "promo_code": "SAVE20"},
    {"items": [{"name": "Tablet", "price": 399.99, "quantity": 1}], "promo_code": "SAVE20"}
]

order_ids = []
for i, order_data in enumerate(bulk_orders, 1):
    order_id = f"BULK-{int(time.time())}-{i:02d}"
    order_ids.append(order_id)
    
    order_message = {
        "order_id": order_id,
        "items": order_data["items"],
        "promo_code": order_data.get("promo_code")
    }
    
    sqs.send_message(QueueUrl=TASK_QUEUE_URL, MessageBody=json.dumps(order_message))
    
    status = "❌ BAD" if order_data["items"][0]["price"] < 0 else "✅"
    print(f"{status} Sent: {order_id} - {order_data['items'][0]['name']}")
    time.sleep(0.1)

print(f"\n📤 Total sent: {len(bulk_orders)} orders")
print("⏳ Waiting 25s for processing + DLQ recovery...")
time.sleep(25)

# Check results
print("\n" + "="*60)
print("📊 RESULTS:")
print("="*60)

success_count = 0
dlq_recovered = 0
failed_count = 0

for order_id in order_ids:
    try:
        obj = s3.get_object(Bucket=BUCKET_NAME, Key=f"{order_id}.json")
        invoice = json.loads(obj['Body'].read())
        
        if invoice.get('recovered_from_dlq'):
            dlq_recovered += 1
            print(f"🔧 {order_id}: ${invoice['final_total']:.2f} (RECOVERED FROM DLQ)")
        else:
            success_count += 1
            print(f"✅ {order_id}: ${invoice['final_total']:.2f}")
    except:
        failed_count += 1
        print(f"❌ {order_id}: Not found")

print("="*60)
print(f"\n📈 Summary:")
print(f"   Total Sent: {len(bulk_orders)}")
print(f"   Processed Successfully: {success_count}")
print(f"   Recovered from DLQ: {dlq_recovered}")
print(f"   Failed: {failed_count}")
print(f"   Success Rate: {((success_count + dlq_recovered)/len(bulk_orders)*100):.1f}%")


📦 Sending 10 Orders (9 good + 1 bad)...
✅ Sent: BULK-1770637800-01 - Laptop
✅ Sent: BULK-1770637800-02 - Monitor
✅ Sent: BULK-1770637801-03 - Keyboard
❌ BAD Sent: BULK-1770637801-04 - Mouse
✅ Sent: BULK-1770637801-05 - Headphones
✅ Sent: BULK-1770637801-06 - Webcam
✅ Sent: BULK-1770637801-07 - Desk Chair
✅ Sent: BULK-1770637801-08 - USB Hub
✅ Sent: BULK-1770637802-09 - Speaker
✅ Sent: BULK-1770637802-10 - Tablet

📤 Total sent: 10 orders
⏳ Waiting 25s for processing + DLQ recovery...

📊 RESULTS:
✅ BULK-1770637800-01: $899.99
✅ BULK-1770637800-02: $479.98
✅ BULK-1770637801-03: $79.99
❌ BULK-1770637801-04: Not found
✅ BULK-1770637801-05: $134.99
✅ BULK-1770637801-06: $85.49
✅ BULK-1770637801-07: $174.99
✅ BULK-1770637801-08: $71.98
✅ BULK-1770637802-09: $63.99
✅ BULK-1770637802-10: $319.99

📈 Summary:
   Total Sent: 10
   Processed Successfully: 9
   Recovered from DLQ: 0
   Failed: 1
   Success Rate: 90.0%


In [122]:
# Cell 6: DLQ DEMO - Fail → Retry → DLQ → Fix → Save → Fetch

import time
import json

# Configure fast retries (5s instead of 30s)
sqs.set_queue_attributes(QueueUrl=TASK_QUEUE_URL, Attributes={'VisibilityTimeout': '5'})

# Send order with negative values (task_lambda will reject, DLQ will fix)
order_id = f"DLQ-TEST-{int(time.time())}"
bad_order = {
    "order_id": order_id,
    "items": [{"name": "Laptop", "price": -999.99, "quantity": -2}],
    "promo_code": "SAVE10"
}

print(f"🧪 DLQ DEMO: {order_id}")
print("="*60)
print(f"💥 Sending order with negative price/quantity...")
sqs.send_message(QueueUrl=TASK_QUEUE_URL, MessageBody=json.dumps(bad_order))

print(f"⏳ Waiting 20s for: fail → retry → DLQ → fix → save...")
time.sleep(20)

print(f"\n📥 Fetching recovered invoice from S3...")
try:
    response = s3.get_object(Bucket=BUCKET_NAME, Key=f"{order_id}.json")
    invoice = json.loads(response['Body'].read())
    
    print(f"✅ SUCCESS - Order recovered by DLQ!\n")
    print(f"📄 Invoice:")
    print(f"   Order ID: {invoice['order_id']}")
    print(f"   Items: {invoice['item_count']}")
    print(f"   Subtotal: ${invoice['subtotal']}")
    print(f"   Discount: ${invoice['discount']}")
    print(f"   Final Total: ${invoice['final_total']}")
    print(f"   Promo Code: {invoice['promo_code']}")
    print(f"   Status: {invoice['status']}")
    print(f"   Recovered from DLQ: {invoice.get('recovered_from_dlq', False)}")
    print(f"   Fixes Applied: {invoice.get('dlq_fixes', [])}")
    
except Exception as e:
    print(f"❌ Invoice not found: {e}")
    print(f"   DLQ may still be processing - wait longer or check logs")

print("="*60)


🧪 DLQ DEMO: DLQ-TEST-1770194988
💥 Sending order with negative price/quantity...
⏳ Waiting 20s for: fail → retry → DLQ → fix → save...

📥 Fetching recovered invoice from S3...
✅ SUCCESS - Order recovered by DLQ!

📄 Invoice:
   Order ID: DLQ-TEST-1770194988
   Items: 1
   Subtotal: $1999.98
   Discount: $200.0
   Final Total: $1799.98
   Promo Code: SAVE10
   Status: completed
   Recovered from DLQ: False
   Fixes Applied: []


In [16]:
order_message = {
    "order_id": order_id,
    "user_id": "test-user-123",  # ← ADD THIS LINE
    "items": [
        {"name": "Laptop", "price": 200.99, "quantity": 1},
        {"name": "Mouse", "price": 30.99, "quantity": 2},
        {"name": "Keyboard", "price": 50.00, "quantity": 1}
    ],
    "promo_code": "SAVE10"
}
